#### Imports

In [2]:
import time
import pandas as pd

from tqdm import tqdm

from nba_api.stats.static import players, teams
from nba_api.stats.endpoints import leaguedashptstats

#### Scraping Functions

In [4]:
def _scrape_traded_touches_totals(season_param, player_id, team, start_date='', end_date=''):
    '''
    returns a dataframe of length one with the total touches a player had for that team during his stint that season using the NBA API
    '''
    tracking_df = leaguedashptstats.LeagueDashPtStats(season=season_param,
                                                      player_or_team='Player',
                                                      pt_measure_type='Possessions',
                                                      per_mode_simple='Totals',
                                                      date_from_nullable=start_date,
                                                      date_to_nullable=end_date,
                                                     ).get_data_frames()[0]
    p_tracking_df = tracking_df[tracking_df['PLAYER_ID'] == player_id].reset_index(drop=True)
    p_tracking_df['SEASON'] = season_param
    p_tracking_df = p_tracking_df[['PLAYER_ID','PLAYER_NAME','TEAM_ABBREVIATION','TIME_OF_POSS','TOUCHES','FRONT_CT_TOUCHES','AVG_DRIB_PER_TOUCH','SEASON']].copy()
    p_tracking_df['TEAM_ABBREVIATION'] = team
    return p_tracking_df
  
def _scrape_touches_total(season_param, filepath, rs):
    '''
    returns nothing, but interacts with the NBA API and saves a dataframe of tracking/time touches possession data in a local directory
    '''
    if rs:
        tracking_df = leaguedashptstats.LeagueDashPtStats(season=season_param,
                                                          player_or_team='Player',
                                                          pt_measure_type='Possessions',
                                                          per_mode_simple='Totals',
                                                         ).get_data_frames()[0]
        tracking_df['SEASON'] = season_param
        tracking_df = tracking_df[['PLAYER_ID','PLAYER_NAME','TEAM_ABBREVIATION','TIME_OF_POSS','TOUCHES','FRONT_CT_TOUCHES','AVG_DRIB_PER_TOUCH','SEASON']].copy()
        tracking_df.to_csv(filepath + '{}_touches_totals.csv'.format(season_param), index=False)
    else:
        tracking_df = leaguedashptstats.LeagueDashPtStats(season=season_param,
                                                          player_or_team='Player',
                                                          pt_measure_type='Possessions',
                                                          per_mode_simple='Totals',
                                                          season_type_all_star='Playoffs'
                                                         ).get_data_frames()[0]
        tracking_df['SEASON'] = season_param
        tracking_df = tracking_df[['PLAYER_ID','PLAYER_NAME','TEAM_ABBREVIATION','TIME_OF_POSS','TOUCHES','FRONT_CT_TOUCHES','AVG_DRIB_PER_TOUCH','SEASON']].copy()
        tracking_df.to_csv(filepath + '{}_touches_playoff_totals.csv'.format(season_param), index=False)
    
def _scrape_traded_seasonal_touches(start_year=2013, end_year=2023, filepath='data/touches/traded-totals/'):
    '''
    returns nothing, but saves a traded player's tracking/time touches possessions data in a local directory
    '''
    traded_df = pd.read_csv('data/ids/traded_players_database.csv')
    traded_tracking_df = pd.DataFrame()
    
    for pid, season, start_date, end_date, team in tqdm(zip(traded_df['PLAYER_ID'].to_list(), traded_df['SEASON'].to_list(),
                                                            traded_df['START_DATE'].to_list(), traded_df['END_DATE'].to_list(),
                                                            traded_df['TEAM'].to_list()
                                                  ), total=len(traded_df)):
        while True:
            try:
                temp_df = _scrape_traded_touches_totals(season, pid, team, start_date, end_date)
                traded_tracking_df = pd.concat([traded_tracking_df, temp_df]).reset_index(drop=True)
                time.sleep(0.65)
                break
            except:
                pass
    
    season_param = str(start_year) + '-' + str(end_year)[2:]
    traded_tracking_df.to_csv(filepath + '{}_traded_touches_totals.csv'.format(season_param), index=False)
    
def _scrape_seasonal_touches(start_year=2013, end_year=2023, filepath='data/touches/season-totals/', rs=True):
    '''
    returns nothing, but saves seasonal tracking/time touches possessions data in a local directory
    '''
    for season in tqdm(range(start_year, end_year)):
        season_param = str(season) + '-' + str(season+1)[2:]
        _scrape_touches_total(season_param, filepath, rs)
        time.sleep(.3)

#### Call the Scraping Functions & Scrape Data

In [13]:
_scrape_seasonal_touches()

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:17<00:00,  1.71s/it]


In [5]:
_scrape_seasonal_touches(filepath='data/touches/playoff-totals/', rs=False)

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:19<00:00,  1.90s/it]


In [6]:
_scrape_traded_seasonal_touches()

100%|██████████████████████████████████████████████████████████████████████████████| 1409/1409 [32:31<00:00,  1.39s/it]
